In [1]:
from keras import backend as k
from keras import applications, optimizers, losses, layers
from keras.utils.training_utils import multi_gpu_model

from keras import layers ,losses
from keras.layers import Dropout, Input, Flatten, Dense, GlobalAveragePooling2D, Activation, TimeDistributed
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.convolutional import Conv2D
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import math
import csv
import os
import tqdm

Using TensorFlow backend.


In [2]:
path = './img_aug_sample/'
list_file = sorted(os.listdir(path))

labels = list([i[:9] for i in list_file])
data_list = list(set([i[:9] for i in list_file]))

keys = ['_OR', '_BN', '_BP', '_GN', '_GP']
degree = ['_0000', '_0090', '_0180', '_0270']

In [3]:
imgs = []
labs = []
for data in data_list:
    for key in keys:
        img = []
        print('======================')
        for deg in degree:
            print(data+deg+key+'.jpg')
            img.append(preprocess_input(cv2.imread(os.path.join(path, data+deg+key+'.jpg'))))
        print('======================')
        imgs.append(img)
        labs.append([int(data.split('_')[0]), int(data.split('_')[1])])
imgs = np.array(imgs)
labs = np.array(labs)

0004_0002_0000_OR.jpg
0004_0002_0090_OR.jpg
0004_0002_0180_OR.jpg
0004_0002_0270_OR.jpg
0004_0002_0000_BN.jpg
0004_0002_0090_BN.jpg
0004_0002_0180_BN.jpg
0004_0002_0270_BN.jpg
0004_0002_0000_BP.jpg
0004_0002_0090_BP.jpg
0004_0002_0180_BP.jpg
0004_0002_0270_BP.jpg
0004_0002_0000_GN.jpg
0004_0002_0090_GN.jpg
0004_0002_0180_GN.jpg
0004_0002_0270_GN.jpg
0004_0002_0000_GP.jpg
0004_0002_0090_GP.jpg
0004_0002_0180_GP.jpg
0004_0002_0270_GP.jpg
0002_0002_0000_OR.jpg
0002_0002_0090_OR.jpg
0002_0002_0180_OR.jpg
0002_0002_0270_OR.jpg
0002_0002_0000_BN.jpg
0002_0002_0090_BN.jpg
0002_0002_0180_BN.jpg
0002_0002_0270_BN.jpg
0002_0002_0000_BP.jpg
0002_0002_0090_BP.jpg
0002_0002_0180_BP.jpg
0002_0002_0270_BP.jpg
0002_0002_0000_GN.jpg
0002_0002_0090_GN.jpg
0002_0002_0180_GN.jpg
0002_0002_0270_GN.jpg
0002_0002_0000_GP.jpg
0002_0002_0090_GP.jpg
0002_0002_0180_GP.jpg
0002_0002_0270_GP.jpg
0005_0007_0000_OR.jpg
0005_0007_0090_OR.jpg
0005_0007_0180_OR.jpg
0005_0007_0270_OR.jpg
0005_0007_0000_BN.jpg
0005_0007_

0005_0002_0090_GP.jpg
0005_0002_0180_GP.jpg
0005_0002_0270_GP.jpg
0003_0004_0000_OR.jpg
0003_0004_0090_OR.jpg
0003_0004_0180_OR.jpg
0003_0004_0270_OR.jpg
0003_0004_0000_BN.jpg
0003_0004_0090_BN.jpg
0003_0004_0180_BN.jpg
0003_0004_0270_BN.jpg
0003_0004_0000_BP.jpg
0003_0004_0090_BP.jpg
0003_0004_0180_BP.jpg
0003_0004_0270_BP.jpg
0003_0004_0000_GN.jpg
0003_0004_0090_GN.jpg
0003_0004_0180_GN.jpg
0003_0004_0270_GN.jpg
0003_0004_0000_GP.jpg
0003_0004_0090_GP.jpg
0003_0004_0180_GP.jpg
0003_0004_0270_GP.jpg
0003_0007_0000_OR.jpg
0003_0007_0090_OR.jpg
0003_0007_0180_OR.jpg
0003_0007_0270_OR.jpg
0003_0007_0000_BN.jpg
0003_0007_0090_BN.jpg
0003_0007_0180_BN.jpg
0003_0007_0270_BN.jpg
0003_0007_0000_BP.jpg
0003_0007_0090_BP.jpg
0003_0007_0180_BP.jpg
0003_0007_0270_BP.jpg
0003_0007_0000_GN.jpg
0003_0007_0090_GN.jpg
0003_0007_0180_GN.jpg
0003_0007_0270_GN.jpg
0003_0007_0000_GP.jpg
0003_0007_0090_GP.jpg
0003_0007_0180_GP.jpg
0003_0007_0270_GP.jpg
0003_0006_0000_OR.jpg
0003_0006_0090_OR.jpg
0003_0006_

0004_0007_0180_GP.jpg
0004_0007_0270_GP.jpg
0002_0006_0000_OR.jpg
0002_0006_0090_OR.jpg
0002_0006_0180_OR.jpg
0002_0006_0270_OR.jpg
0002_0006_0000_BN.jpg
0002_0006_0090_BN.jpg
0002_0006_0180_BN.jpg
0002_0006_0270_BN.jpg
0002_0006_0000_BP.jpg
0002_0006_0090_BP.jpg
0002_0006_0180_BP.jpg
0002_0006_0270_BP.jpg
0002_0006_0000_GN.jpg
0002_0006_0090_GN.jpg
0002_0006_0180_GN.jpg
0002_0006_0270_GN.jpg
0002_0006_0000_GP.jpg
0002_0006_0090_GP.jpg
0002_0006_0180_GP.jpg
0002_0006_0270_GP.jpg


In [4]:
img_width, img_height = 300, 400
vgg16_model = applications.ResNet50(weights= 'imagenet', include_top=False, input_shape = (img_height, img_width,3))
x = GlobalAveragePooling2D()(vgg16_model.output)

vggmodel = Model(inputs = vgg16_model.input, outputs = x)
vggmodel.trainable = False

KeyboardInterrupt: 

In [ ]:
inputs = Input(shape=(4,400,300,3))

x = TimeDistributed(vggmodel)(inputs)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(2)(x)
x = layers.ReLU(max_value=6)(x)

model = Model(inputs = inputs, outputs = x)

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
hist = model.fit(imgs,labs,batch_size= 4,epochs =3)

In [ ]:
model.evaluate(imgs,labs)

In [ ]:
model.predict(imgs[[20]])

In [ ]:
labs[20]